# Genetic Algorithm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from matplotlib.widgets import Cursor
%matplotlib qt

In [ ]:
np.array(range(1,16))

In [ ]:
# df = pd.read_excel("15-Points.xlsx")
# df['test']= np.array(range(1,16))
# df

In [ ]:
def input_points ():
    points = []

    # display a plot and wait for the user to click on num_points points
    fig, ax = plt.subplots()
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.show()

    cursor = Cursor(ax, horizOn=True, vertOn=True, useblit=True, color='black', linewidth=1)

    # take input fro
    while True:
        # get the first tuple (first point)
        point = plt.ginput(1)
        if point:
            point=point[0]
        else:
            break

        points.append(point)
        plt.plot(point[0], point[1], 'bo')
        plt.draw()
    return points

In [ ]:
def plot_tsp(data, Best_solution,Best_cost):
    x = data['x']
    y = data['y']
    plt.scatter(x, y, color = 'b')
    plt.title('Genetic Algorithm')
    x_init, y_init = data[data['City']==Best_solution[0]]['x'], data[data['City']==Best_solution[0]]['y']
    plt.scatter(x_init, y_init, color = 'red')
    plt.text(x_init - 0.5 , y_init + 0.6 , "Start" , color = 'm')

    x = []
    y = []
    x.append(x_init)
    y.append(y_init)
    TSP_plot = plt.plot(x, y, color = 'r')

    for i, city in enumerate(Best_solution[0:]):
        city_x, city_y = data[data['City']==Best_solution[i]]['x'], data[data['City']==Best_solution[i]]['y']
        x.append(city_x)
        y.append(city_y)  
        plt.setp(TSP_plot, xdata = x, ydata = y)
        plt.pause(0.3)

    x.append(x_init)
    y.append(y_init)
    plt.setp(TSP_plot, xdata = x, ydata = y)
    
    plt.pause(0.3)
#     plt.text(0.5 , 0.6 , f"Best_cost: {Best_cost}" , color = 'm')
    

In [ ]:
def euclidean_distance(df,c1,c2):
    x1 = df.loc[df['City'] == c1, 'x'].iloc[0]
    y1 = df.loc[df['City'] == c1, 'y'].iloc[0]
    x2 = df.loc[df['City'] == c2, 'x'].iloc[0]
    y2 = df.loc[df['City'] == c2, 'y'].iloc[0]
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [ ]:
def distance(df):
    count = len(df)
    dist_arr = np.zeros((count,count))
    for c1 in range(0,count-1):
        for c2 in range(c1+1,count):
            dist_arr[c1,c2]= euclidean_distance(df,c1,c2)
            dist_arr[c2,c1] =dist_arr[c1,c2]

        dist_arr[c1,c1] = np.inf
    dist_arr[c1+1,c1+1] = np.inf

    return dist_arr

In [ ]:
class Chromosome :
    def __init__(self,chrom):
        self.chrom = chrom
        self.distList = []
        self.cost = None

    def evaluateFitness(self,dist_arr) :
        self.distList = []
        for i in range(0,len(self.chrom)-1) :
            dist = dist_arr[self.chrom[i]][self.chrom[i+1]]
            self.distList.append(dist)
        self.cost = sum(self.distList)




In [ ]:
#uniform Crossover
def Crossover(crossover_list,count,dist_arr):
    uniform = random.choices([0,1], k=count)
    newgen = [None]*len(crossover_list)
    for c in range(0,len(crossover_list),2):
        newgen[c]=Chromosome(crossover_list[c].chrom)
        newgen[c+1]=Chromosome(crossover_list[c].chrom)
        for i in range(len(uniform)) :
            if uniform[i] == 1:
                newgen[c].chrom[i],newgen[c+1].chrom[i] = newgen[c+1].chrom[i] , newgen[c].chrom[i]
        newgen[c].evaluateFitness(dist_arr)
        newgen[c+1].evaluateFitness(dist_arr)

    return newgen



# Mutation

def Swapping(chrom):
    sw_ndx = random.sample(range(len(chrom)), k=2)
    newChrom= chrom.copy()
    newChrom[sw_ndx[0]],newChrom[sw_ndx[1]] = newChrom[sw_ndx[1]] ,newChrom[sw_ndx[0]]
    return newChrom

def Scramble(chrom):
    left = random.randint(0, len(chrom)-2)
    right = random.randint(left+1, len(chrom))
    newChrom= chrom.copy()
    new = random.sample(newChrom[left : right], len(newChrom[left : right]))
    newChrom = np.array(newChrom)
    newChrom[left : right] = new
    newChrom= newChrom.tolist()
    return newChrom

def Inversion(chrom):
    left = random.randint(0, len(chrom)-2)
    right = random.randint(left+1, len(chrom))
    newChrom= chrom.copy()
    new = newChrom[left : right]
    new.reverse()
    newChrom = np.array(newChrom)
    newChrom[left : right] = new
    newChrom= newChrom.tolist()
    return newChrom

def Mutation (M_list,dist_arr):
    newgen = [None]*len(M_list)
    for i in range(len(M_list)) :
        newgen[i] = Chromosome(M_list[i].chrom.copy())
        r = random.randint(0, 3)
        if r == 0:
            newgen[i].chrom = Swapping(newgen[i].chrom).copy()
        elif r == 1:
            newgen[i].chrom = Scramble(newgen[i].chrom).copy()
        else:
            newgen[i].chrom = Inversion(newgen[i].chrom).copy()

        newgen[i].evaluateFitness(dist_arr)

    return newgen



In [ ]:
Inversion([3,5,6,7,8,4])

In [ ]:
#########################  Genetic Algorithm ##################################

In [ ]:
def Genetic(pop_size, gen_count, elit, cross_num , mut_num):
    
    points = input_points()
    data = pd.DataFrame(points, columns = ['x', 'y'])
    data['City'] = np.random.permutation(len(points))
#     data = pd.read_excel("15-Points.xlsx")
    #distance
    dist_arr = distance(data)
    count = len(data)
    
    #Initial Population 
    city_n=[i for i in range(0,count)]
    pop_list = [None]*pop_size
    for p in range(pop_size): #Generate Chromosome 
        chrom = random.sample(city_n, k=len(data))
        pop_list[p] = Chromosome(chrom)
        pop_list[p].evaluateFitness(dist_arr)
    #sort
    pop_list = sorted(pop_list, key=lambda d: d.cost)
    
    # Initialization Generations
    for i in range(gen_count):
        next_pop = []

        #select elitism
        elit_list = pop_list[0:elit]

        # Perform crossover
        select_crossover = random.sample(pop_list, k=cross_num)
        crossover_list = Crossover(select_crossover,count,dist_arr).copy()

        # Perform mutation
        select_Mutation = random.sample(crossover_list.copy(), k=mut_num)
        Mutation_list = Mutation(select_Mutation,dist_arr).copy()

        # select Population size
        next_pop =  elit_list + pop_list + crossover_list + Mutation_list
        next_pop = sorted(next_pop, key=lambda d: d.cost )

        pop_list = next_pop[0:pop_size]

    # Best solution
#     print("Best Path : ",pop_list[0].chrom)
#     print("Path dist : ", np.round(pop_list[0].distList , 3).tolist())
#     print("Best cost : ",pop_list[0].cost)
    
    Best_Path = pop_list[0].chrom.copy()
    Path_dist = np.round(pop_list[0].distList.copy() , 3).tolist()
    Best_cost = pop_list[0].cost
    
    plot_tsp(data, Best_Path,Best_cost)
    
    
    return Best_Path, Path_dist,Best_cost

In [70]:
#Genetic algorithm parameters
pop_size= 100 
gen_count=200 
elit = 3
cross_num = 30
mut_num = 8

Best_Path, Path_dist,Best_cost= Genetic(pop_size, gen_count, elit, cross_num , mut_num)

In [ ]:
#########################  test code  ##################################

In [ ]:
pop_size= 50 
gen_count=150 
elit = 3
cross_num = 30
mut_num = 8

points = input_points()
count = len(points)
df = pd.DataFrame(points, columns = ['x', 'y'])

df['City'] = np.array(range(0,count))
# df = pd.read_excel("15-Points.xlsx")
city_n=[i for i in range(0,count)]

#distance
dist_arr = distance(df)

#Initial Population  
pop_list = [None]*pop_size
for p in range(pop_size):
    #Generate Chromosome 
    chrom = random.sample(city_n, k=count)
    pop_list[p] = Chromosome(chrom)
    pop_list[p].evaluateFitness(dist_arr)
    # print(pop_list[p].cost)


pop_list = sorted(pop_list, key=lambda d: d.cost )


# Initialization Generations
for i in range(gen_count):
    next_pop = []

    #select elitism
    elit_list = pop_list[0:elit]

    # Perform crossover
    select_crossover = random.sample(pop_list, k=cross_num)
    crossover_list = Crossover(select_crossover,count,dist_arr).copy()

    # Perform mutation
    select_Mutation = random.sample(crossover_list.copy(), k=mut_num)
    Mutation_list = Mutation(select_Mutation,dist_arr).copy()
    
    # select Population size
    next_pop =  elit_list + pop_list + crossover_list + Mutation_list
    next_pop = sorted(next_pop, key=lambda d: d.cost )

    # next_pop = list(set(next_pop))
    # print(next_pop[0].cost)
    pop_list = next_pop[0:pop_size]


print("Best Path : ",pop_list[0].chrom)
print("Path dist : ", np.round(pop_list[0].distList , 3).tolist())
print("Best cost : ",pop_list[0].cost)
Best_Path = pop_list[0].chrom.copy()

plot_tsp(df, Best_Path)
